# Using the OpenAI Client with Whisk

This notebook will walk you through examples of how you can interact with whisk using the OpenAI client.

We will cover the following topics:
- Chat Completions
- File Uploads
- Retrieving Files
- Deleting Files

In [1]:
%pip install nest_asyncio


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import nest_asyncio


In [3]:
nest_asyncio.apply()


# Endpoint 

point the base_url to the whisk server

In [4]:
from openai import OpenAI

client = OpenAI(base_url="http://localhost:8000/v1")

# Chat Completions

model is your whisk app in the following format @app-name-version/handler-name

metadata is a dictionary of key-value pairs that will be added to the request

In [5]:



response = client.chat.completions.create(
    model="@storage-example-0.0.1/default",
    messages=[{"role": "user", "content": "whats the most important part of the readme?"}],
    metadata={"user_id": "123"},
)

print(response)


ChatCompletion(id='chatcmpl-1739769080', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The most important part of a README file is typically the introduction or overview section. This section should provide a brief summary of the project, its purpose, and any key information that users or contributors need to know. It sets the tone for the rest of the document and helps readers quickly understand what the project is about and how they can get started.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1739769080, model='@whisk-example-app-0.0.1/chat.completions', object='chat.completion', service_tier=None, system_fingerprint=None, usage=None, metadata=None)


# Storage With Custom Storage

# File Uploads with FileExtraBody

The FileExtraBody class allows you to specify additional parameters when uploading files:
- model: Target a specific storage handler using format "@app-name-version/handler-name" 
- metadata: Additional metadata as a comma-separated string of key=value pairs

Examples:
1. Using custom storage handler:
   file_extra = FileExtraBody(
       model="@storage-example-0.0.1/custom-storage",
       metadata="user_id=123,purpose=chat"
   )

2. Default storage handler:
   No extra body needed, just use client.files.create() directly

The client.files API supports:
- create(): Upload new files
- list(): List all files
- retrieve(): Get file details
- delete(): Remove files


In [27]:
import json

from whisk.kitchenai_sdk.http_schema import FileExtraBody   
#targeting custom storage handler
file_extra_body = FileExtraBody(
    model="@storage-example-0.0.1/custom-storage",
    metadata="user_id=123,other_key=value"  # Changed to string format
)

response = client.files.create(
    file=open("README.md", "rb"),
    purpose="chat",
    extra_body=file_extra_body.model_dump()
)
print(response)

FileObject(id='file-file-1739906089', bytes=10340, created_at=1739906089, filename='README.md', object='file', purpose='chat', status='processed', status_details=None)


# Get Requests

using the same extra body as above for query fields on get requests

In [29]:
file_extra_query = FileExtraBody(
    model="@storage-example-0.0.1/custom-storage",
    metadata="user_id=123,other_key=value"  # Changed to string format
)

client.files.list(extra_query=file_extra_query.model_dump())

SyncCursorPage[FileObject](data=[FileObject(id='file-file-1739904468', bytes=10340, created_at=1739904468, filename='file-1739904468', object='file', purpose='fine-tune', status='processed', status_details=None), FileObject(id='file-file-1739905883', bytes=10340, created_at=1739905883, filename='file-1739905883', object='file', purpose='fine-tune', status='processed', status_details=None), FileObject(id='file-file-1739906089', bytes=10340, created_at=1739906089, filename='file-1739906089', object='file', purpose='fine-tune', status='processed', status_details=None), FileObject(id='file-file-1739905460', bytes=10340, created_at=1739905460, filename='file-1739905460', object='file', purpose='fine-tune', status='processed', status_details=None), FileObject(id='file-file-1739905405', bytes=10340, created_at=1739905405, filename='file-1739905405', object='file', purpose='fine-tune', status='processed', status_details=None)], has_more=False, object='list', first_id='file-file-1739904468', la

In [41]:
client.files.list(extra_query=file_extra_query.model_dump())

SyncCursorPage[FileObject](data=[FileObject(id='file-1739905883', bytes=10340, created_at=1739905883, filename='file-1739905883', object='file', purpose='fine-tune', status='processed', status_details=None), FileObject(id='file-1739906525', bytes=10340, created_at=1739906525, filename='file-1739906525', object='file', purpose='fine-tune', status='processed', status_details=None), FileObject(id='file-1739906089', bytes=10340, created_at=1739906089, filename='file-1739906089', object='file', purpose='fine-tune', status='processed', status_details=None), FileObject(id='file-1739905460', bytes=10340, created_at=1739905460, filename='file-1739905460', object='file', purpose='fine-tune', status='processed', status_details=None), FileObject(id='file-1739905405', bytes=10340, created_at=1739905405, filename='file-1739905405', object='file', purpose='fine-tune', status='processed', status_details=None)], has_more=False, object='list', first_id='file-1739905883', last_id='file-1739905405', after

In [53]:
client.files.retrieve("file-1739905883", extra_query=file_extra_query.model_dump())

FileObject(id='file-1739905883', bytes=10340, created_at=1739905883, filename='file-1739905883', object='file', purpose='fine-tune', status='processed', status_details=None)

In [56]:
client.files.delete("file-1739905883", extra_query=file_extra_query.model_dump())


FileDeleted(id='file-1739905883', deleted=True, object='file')

# Targeting default storage handler

You don't need to specify extra file params for each request. 

It will default use any handler that is name "storage" as the default handler. Letting you use a simpler client request.

In [30]:
response = client.files.create(
    file=open("README.md", "rb"),
    purpose="chat",
)
print(response)

FileObject(id='file-file-1739906525', bytes=10340, created_at=1739906525, filename='README.md', object='file', purpose='chat', status='processed', status_details=None)


In [38]:
client.files.list()


SyncCursorPage[FileObject](data=[FileObject(id='file-1739904468', bytes=10340, created_at=1739904468, filename='file-1739904468', object='file', purpose='fine-tune', status='processed', status_details=None), FileObject(id='file-1739905883', bytes=10340, created_at=1739905883, filename='file-1739905883', object='file', purpose='fine-tune', status='processed', status_details=None), FileObject(id='file-1739906525', bytes=10340, created_at=1739906525, filename='file-1739906525', object='file', purpose='fine-tune', status='processed', status_details=None), FileObject(id='file-1739906089', bytes=10340, created_at=1739906089, filename='file-1739906089', object='file', purpose='fine-tune', status='processed', status_details=None), FileObject(id='file-1739905460', bytes=10340, created_at=1739905460, filename='file-1739905460', object='file', purpose='fine-tune', status='processed', status_details=None), FileObject(id='file-1739905405', bytes=10340, created_at=1739905405, filename='file-17399054

In [39]:
client.files.retrieve("file-1739904468")


FileObject(id='file-1739904468', bytes=10340, created_at=1739904468, filename='file-1739904468', object='file', purpose='fine-tune', status='processed', status_details=None)

In [40]:
client.files.delete("file-1739904468")


FileDeleted(id='file-1739904468', deleted=True, object='file')